**Purpose**

The purpose of this test is to evaluate analytical abilities. A case study is an integral part while selecting candidates for an engineering post. It allows the candidates to experience situations that they might actually face in the current world. When responding to the questions, add the source code that is used to derive the answer.

**Scenario:**
1. Suppose You have a JSON file: **pos_0.png.json**. Your job is to write a python script that will read JSON from a folder and convert this format into a standard format: **formatted_pos_0.png.json**

--> note: object attribute (presence) will be dependent on if the class (vehicle or license plate) is present on the raw JSON. If both the classes are missing look at the example **formatted_pos_10492.png.json**

You may test out your code with the following jsons:

**Pos_0.png.json**

**Pos_10010.png.json**

**Pos_10492.png.json**

2. Write a python that can read json from a folder and combine the test jsons (**Pos_0.png.json**,**Pos_10010.png.json**, **Pos_10492.png.json**) into a single json file and then change the class names into vehicle>car, license plate>number

In [1]:
!mkdir /content/sampleJson
# upload files if not available
!mkdir /content/sampleJson/scenario1
!mkdir /content/sampleJson/scenario2

In [3]:
%cd /content/sampleJson
!cp pos_0.png.json pos_10010.png.json pos_10492.png.json /content/sampleJson/scenario1/
!cp pos_0.png.json pos_10010.png.json pos_10492.png.json /content/sampleJson/scenario2/

/content/sampleJson


In [4]:
import os
import glob
import json

Answer to the scenario 1.

In [5]:
def scenario1():
    scenario1_content_path = "/content/sampleJson/scenario1/"
    file_extension = "json"
    os.chdir(scenario1_content_path)
    file_list = glob.glob('*.{}'.format(file_extension))
    
    for file_name in file_list:
        file_path = '{}{}'.format(scenario1_content_path, file_name)
        findings = {
            "dataset_name": str(file_name),
            "image_link": "",
            "annotation_type": "image",
            "annotation_objects": {
                "vehicle": {
                    "presence": 0,
                    "bbox": []
                },
                "license_plate": {
                    "presence": 0,
                    "bbox": []
                }
            },
            "annotation_attributes": {
                "vehicle": {
                    "Type": None,
                    "Pose": None,
                    "Model": None,
                    "Make": None,
                    "Color": None
                },
                "license_plate": {
                    "Difficulty Score": None,
                    "Value": None,
                    "Occlusion": None
                }
            }
        }
        vehicle_presence = 0
        license_plate_presence = 0
        # Opening JSON file
        with open(file_path) as f:
            data_dict = json.load(f)
            objects = data_dict["objects"]
            if len(objects) > 0:
                for obj in objects:
                    if obj["classTitle"] == "Vehicle":
                        vehicle_presence = vehicle_presence + 1
                        tags = obj["tags"]
                        for tag in tags:
                            findings["annotation_attributes"]["vehicle"][str(tag["name"])] = str(tag["value"])
                        exterior = obj["points"]["exterior"]
                        if len(exterior) > 0:
                            exterior_list = []
                            for value in exterior:
                                exterior_list = exterior_list + value
                            findings["annotation_objects"]["vehicle"]["bbox"] = exterior_list
                    elif obj["classTitle"] == "License Plate":
                        license_plate_presence = license_plate_presence + 1
                        findings["annotation_attributes"]["license_plate"]["Occlusion"] = 0
                        tags = obj["tags"]
                        for tag in tags:
                            findings["annotation_attributes"]["license_plate"][str(tag["name"])] = int(tag["value"]) if str(tag["value"]).isdigit() else str(tag["value"])
                        exterior = obj["points"]["exterior"]
                        if len(exterior) > 0:
                            exterior_list = []
                            for value in exterior:
                                exterior_list = exterior_list + value
                            findings["annotation_objects"]["license_plate"]["bbox"] = exterior_list
                    findings["annotation_objects"]["vehicle"]["presence"] = int(vehicle_presence)
                    findings["annotation_objects"]["license_plate"]["presence"] = int(license_plate_presence)
                result = [findings]
            else:
                result = [findings]
            formatted_file_name = "formatted_" + file_name
            formatted_file_path = '{}{}'.format(scenario1_content_path, formatted_file_name)
            with open(formatted_file_path, mode="w") as ff:
                json.dump(result, ff, indent=4)

In [7]:
scenario1()

Answer to the scenario 2.

In [6]:
def scenario2():
    scenario2_content_path = "/content/sampleJson/scenario2/"
    file_extension = "json"
    os.chdir(scenario2_content_path)
    file_list = glob.glob('*.{}'.format(file_extension))
    accumulated_json_list =[]
    
    for file_name in file_list:
        file_path = '{}{}'.format(scenario2_content_path, file_name)
        # Opening JSON file
        with open(file_path) as f:
            data_dict = json.load(f)
            try:
                data_dict["objects"][0]["classTitle"] = "Car" if data_dict["objects"][0]["classTitle"] == "Vehicle" else "Number"
                data_dict["objects"][1]["classTitle"] = "Number" if data_dict["objects"][0]["classTitle"] == "License Plate" else "Car"
            except:
              pass
            # accumulated_json.update(data_dict)
            accumulated_json_list.append(data_dict)
    formatted_file_name = "accumulated_pos.png.json"
    formatted_file_path = '{}{}'.format(scenario2_content_path, formatted_file_name)
    with open(formatted_file_path, mode="w") as ff:
        json.dump(accumulated_json_list, ff, indent=4)

In [8]:
scenario2()

In [ ]:
if __name__ == '__main__':
    try:
        scenario1()
        scenario2()
    except Exception as e:
        print("Encountered exception {}".format(e))
